# Libraries

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.activations import relu
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob as gb
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
import re
from keras.preprocessing.image import img_to_array
from tqdm import tqdm
from PIL import Image

**Path**

In [ ]:
x_train_path = '/kaggle/input/landscape-image-colorization/landscape Images/gray'
y_train_path = '/kaggle/input/landscape-image-colorization/landscape Images/color'


# **Pre-Processing**

In [ ]:
gray_img=[]
color_img=[]
for img in os.listdir(x_train_path):
    image_path = os.path.join(x_train_path,img)
    g=Image.open(image_path)
    new_size = (224,224)
    resized_img = g.resize(new_size)
    gray_img.append(resized_img)
for img in os.listdir(y_train_path):
    image_path = os.path.join(y_train_path,img)
    g=Image.open(image_path)
    new_size = (224, 224)
    resized_img = g.resize(new_size)
    color_img.append(resized_img)


In [ ]:
gray_img=np.array(gray_img)
color_img=np.array(color_img)
gray_img=np.expand_dims(gray_img,axis=-1)
gray_img=gray_img/255
color_img=color_img/255


In [ ]:
train_gray_img=gray_img[:6000]
test_gray_img=gray_img[6000:]
train_color_img=color_img[:6000]
test_color_img=color_img[6000:]
print(train_gray_img.shape)
print(test_gray_img.shape)
print(train_color_img.shape)
print(test_color_img.shape)

# visualize

In [ ]:
def plot_images(color,grayscale):
    plt.figure(figsize=(15,15))
    plt.subplot(1,3,1)
    plt.title('Color Image', color = 'green', fontsize = 20)
    plt.imshow(color)
    plt.subplot(1,3,2)
    plt.title('Grayscale Image ', color = 'black', fontsize = 20)
    plt.imshow(grayscale,cmap='gray')

    plt.show()
for i in range(3,10):
     plot_images(train_color_img[i],train_gray_img[i])

# **Model**

In [ ]:
def u_net(h=(224,224,1)):
  gg=layers.Input(shape=h)
  x= gg
  conv =[]
  f=64

  #down_sampling
  for i in range (4):
    x= layers.Conv2D(f,3,activation='relu',padding='same')(x)
    x= layers.Conv2D(f,3,activation='relu',padding='same')(x)
    x= layers.Conv2D(f,3,activation='relu',padding='same')(x)
    x = layers.Dropout(0.3)(x)
    x= layers.BatchNormalization()(x)
    conv.append(x)
    x= layers.MaxPooling2D()(x)
    f*=2


  #bottle-neck
  x= layers.Conv2D(f,3,activation='relu',padding='same')(x)
  x= layers.Conv2D(f,3,activation='relu',padding='same')(x)
  x= layers.Conv2D(f,3,activation='relu',padding='same')(x)
  x= layers.BatchNormalization()(x)
  x = layers.Dropout(0.5)(x)
  ff=512
  j= len(conv)-1
  #up-sampling
  for i in range(4):
    x=layers.Conv2DTranspose(ff,2,strides=(2,2),padding='same')(x)
    x=layers.Concatenate(axis=3)([x,conv[j]])
    x= layers.Conv2D(ff,3,activation='relu',padding='same')(x)
    x= layers.Conv2D(ff,3,activation='relu',padding='same')(x)
    ff//=2
    j-=1


  x = layers.Conv2D(64,2,padding='same',activation='relu')(x)
  output = layers.Conv2D(3,1,padding='same')(x)
  model = models.Model(inputs=gg,outputs=output)
  return model

In [ ]:
model = u_net()
model.summary()
model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mse'])
model.fit(train_gray_img,train_color_img,batch_size=32,epochs=5)

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_68 (Conv2D)  │ (None, 224, 224,  │        640 │ input_layer_4[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_69 (Conv2D)  │ (None, 224, 224,  │     36,928 │ conv2d_68[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_70 (Conv2D)  │ (None, 224, 224,  │     36,928 │ conv2d_69[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 224, 224,  │          0 │ conv2d_70[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 224, 224,  │        256 │ dropout[0][0]     │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_16    │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_71 (Conv2D)  │ (None, 112, 112,  │     73,856 │ max_pooling2d_16… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_72 (Conv2D)  │ (None, 112, 112,  │    147,584 │ conv2d_71[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_73 (Conv2D)  │ (None, 112, 112,  │    147,584 │ conv2d_72[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 112, 112,  │          0 │ conv2d_73[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        512 │ dropout_1[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_17    │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_74 (Conv2D)  │ (None, 56, 56,    │    295,168 │ max_pooling2d_17… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_75 (Conv2D)  │ (None, 56, 56,    │    590,080 │ conv2d_74[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_76 (Conv2D)  │ (None, 56, 56,    │    590,080 │ conv2d_75[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 56, 56,    │          0 │ conv2d_76[0][0] 

 Total params: 43,627,715 (166.43 MB)

 Trainable params: 43,623,747 (166.41 MB)

 Non-trainable params: 3,968 (15.50 KB)

In [ ]:
predicted=model.predict(test_gray_img)


In [ ]:
def plot_images(color,grayscale,predicted):
    plt.figure(figsize=(15,15))
    plt.subplot(1,3,1)
    plt.title('Color Image', color = 'green', fontsize = 20)
    plt.imshow(color)
    plt.subplot(1,3,2)
    plt.title('Grayscale Image ', color = 'black', fontsize = 20)
    plt.imshow(grayscale,cmap='gray')
    plt.subplot(1,3,3)
    plt.title('Predicted Image ', color = 'Red', fontsize = 20)
    plt.imshow(predicted)

    plt.show()

In [ ]:
for i in range(3,10):
     plot_images(test_color_img[i],test_gray_img[i],predicted[i])

# **Save Model**

In [ ]:
import pickle
with open('/kaggle/working/keras_model.pkl', 'wb') as file:
    pickle.dump(model, file)

# **Train More**

In [ ]:
import pickle
with open('/kaggle/working/keras_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)


In [ ]:
loaded_model.fit(train_gray_img,train_color_img,batch_size=32,epochs=2)

In [ ]:
import pickle
with open('/kaggle/working/keras_model.pkl', 'wb') as file:
    pickle.dump(loaded_model, file)